In [1]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

# Basic autograd example 1

In [2]:
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

y = w * x + b

y.backward()

print(x.grad, w.grad, b.grad)  # dx = w, dw = x, db = 1

tensor(2.) tensor(1.) tensor(1.)


# Basic autograd example 2

In [3]:
x = torch.randn(10, 3)
y = torch.randn(10, 2)

linear = nn.Linear(3, 2)
print(f'w: {linear.weight}')
print(f'b: {linear.bias}')

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

pred = linear(x)

loss = criterion(pred, y)
print(f'loss: {loss.item()}')

loss.backward()

print(f'dL/dw: {linear.weight.grad}')
print(f'dL/db: {linear.bias.grad}')

optimizer.step()

pred = linear(x)
loss = criterion(pred, y)

print(f'loss after 1 step optimization: {loss.item()}')

w: Parameter containing:
tensor([[-0.4562, -0.3890,  0.2719],
        [ 0.4773,  0.4494, -0.0940]], requires_grad=True)
b: Parameter containing:
tensor([-0.1084,  0.1605], requires_grad=True)
loss: 1.4161956310272217
dL/dw: tensor([[-0.6673, -0.7450,  0.4259],
        [ 0.0470,  0.6415, -0.4784]])
dL/db: tensor([ 0.3184, -0.1971])
loss after 1 step optimization: 1.3967350721359253


# Loading data from numpy

In [4]:
x = np.array([[1, 2], [3, 4]])
y = torch.from_numpy(x)
z = y.numpy()

# Input pipeline

In [5]:
from turtle import down


train_dataset = torchvision.datasets.CIFAR10(root='D:',
                                            train=True,
                                            transform=transforms.ToTensor(),
                                            download=False)

image, label = train_dataset[0]
print(image.size())
print(label)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                        batch_size=64,
                                        shuffle=True)

data_iter = iter(train_loader)
images, labels = data_iter.next()
for images, labels in train_loader:
    pass

torch.Size([3, 32, 32])
6


# Input pipeline for custom dataset

In [6]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # initialize file paths or list of file names
        pass
    
    def __getitem__(self, index):
        # read one dat from file (np.fromfile, PIL.Image.open ...)
        # preprocess the data (torchvision.transform)
        # Return a data pair (image and label)
        pass
    
    def __len__(self):
        # you should replace 1 to return the length of dataset
        return 1

custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                            batch_size=64,
                                            shuffle=True)

# Pretrained mode

In [7]:
resnet = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)

for param in resnet.parameters():
    param.requires_grad = False

resnet.fc = nn.Linear(resnet.fc.in_features, 100)

images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print(outputs.size())

torch.Size([64, 100])


# Save and load the mode

In [8]:
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

torch.save(resnet.state_dict(), 'param.ckpt')
resnet.load_state_dict(torch.load('param.ckpt'))

<All keys matched successfully>